In [3]:
import pandas as pd
import csv
import networkx as nx
from neo4j import GraphDatabase
from neo4j.graph import Node, Relationship

from vigor import Graph

ImportError: cannot import name 'Graph' from partially initialized module 'vigor.graph' (most likely due to a circular import) (/Users/sjoerdvink/Developer/Experiments/vigor-visualization-recommendation-for-graphs/vigor/graph.py)

In [ ]:
NEO4J_CONNECTION_URI="neo4j+s://demo.neo4jlabs.com:7687"

csv_filename = "../data/use-cases.csv"

In [ ]:
df = pd.read_csv("/content/text2cypher_claudeopus.csv")
df = df[df['syntax_error'] == False]
df = df[df['returns_results'] == True]
df.drop(columns=['timeout', 'syntax_error', 'returns_results', 'false_schema'], inplace=True)

In [ ]:
recommendations = df[df['database'] == "recommendations"]
twitter = df[df['database'] == "twitter"]

In [ ]:
queries = {
    "recommendations": [
      {
        "q_type": "Retrieval",
        "question": "What are the names of the top 5 movies with a budget over 100 million dollars?",
        "query": "MATCH (m:Movie) WHERE m.budget > 100000000 RETURN m"
      },
      {
        "q_type": "Aggregation",
        "question": "What are the top 5 movies by revenue released in 1995?",
        "query": "MATCH (m:Movie) WHERE m.year = 1995 AND m.revenue IS NOT NULL RETURN m"
      },
      {
        "q_type": "Pathfinding",
        "question": "What are the names of actors who have acted in more than five movies?",
        "query": "MATCH (a:Actor)-[:ACTED_IN]->(m:Movie) WITH a, count(m) AS numMovies WHERE numMovies > 5 RETURN a"
      },
       {
        "q_type": "Verbose",
        "question": "Find the names of the top 5 movies with the lowest imdbRating.",
        "query": "MATCH (m:Movie) WHERE m.imdbRating IS NOT NULL RETURN m"
      }
    ],
    "twitter": [
      {
        "q_type": "Retrieval",
        "question": "List the top 3 hashtags used in tweets posted by users from 'Graphs Are Everywhere.",
        "query": "MATCH (u:User {location: 'Graphs Are Everywhere'})-[:POSTS]->(t:Tweet)-[:TAGS]->(h:Hashtag) RETURN h"
      },
      {
        "q_type": "Aggregation",
        "question": "Which three users have tweeted the most as of the latest date in the dataset?",
        "query": "MATCH (u:User)-[:POSTS]->(t:Tweet) WHERE t.created_at IS NOT NULL WITH u, count(t) AS num_tweets, max(t.created_at) AS latest_tweet RETURN u"
      },
      {
        "q_type": "Pathfinding",
        "question": "Which tweets mention the user with the screen name 'neo4j'?",
        "query": "MATCH (t:Tweet)-[:MENTIONS]->(u:User {screen_name: 'neo4j'}) RETURN t"
      },
       {
        "q_type": "Verbose",
        "question": "Find the first 3 tweets that have been retweeted more than 100 times.",
        "query": "MATCH (t:Tweet) WHERE t.favorites > 100 RETURN t"
      }
    ],
}

In [ ]:
header_written = False

for database in queries:
  driver = GraphDatabase.driver(NEO4J_CONNECTION_URI, auth=(database, database))
  for query in queries[database]:
    G = Graph()  # Assuming you are using NetworkX here

    # Execute the query
    records, summary, keys = driver.execute_query(query['query'], database_=database)

    # Process the records and build the graph
    for record in records:
      for element in record:
          if isinstance(element, Node):
              print(f"Node: {element.labels}, Properties: {element.items()}")
              G.add_node(record[0], label=element.labels, **dict(element.items()))  # Convert dict_items to dict
          elif isinstance(element, Relationship):
              print(f"Relationship: {element.type}, Properties: {element.items()}")
              G.add_edge(record[0], record[2], type=element.type, **dict(element.items()))

    # Get graph statistics (ensure G.get_statistics() is implemented)
    graph_stats = G.get_statistics()  # Replace this with however you get statistics from G
    graph_stats['database_name'] = database
    graph_stats['question'] = query['question']
    graph_stats['query'] = query['query']
    graph_stats['q_type'] = query['q_type']

    # Open the CSV file in append mode
    with open(csv_filename, mode='a', newline='') as file:  # Append mode 'a'
        writer = csv.writer(file)

        # Write header only the first time
        if not header_written:
            writer.writerow(graph_stats.keys())
            header_written = True

        # Write the graph statistics for the current query
        writer.writerow(graph_stats.values())
        print('wrote to csv', database)